<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/A_Journey_into_the_Depths_of_Neural_Networks/blob/master/A_Neuron_Dancing_in_Logistic_Regression_Style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Neuron Dancing in Logistic Regression Style**

## **Libraries**

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

## **Loading Dataset and Normalization**

In [2]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


## **Relabeling Data**

In [3]:
# Label 0 is for airplane
y_train = np.where(y_train == 0, 0, 1)
y_test = np.where(y_test == 0, 0, 1)

print("y_train:")
print(y_train[160:170])

print("y_test:")
print(y_test[160:170])

y_train:
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]]
y_test:
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]]


## **Flatten and Reshape Images**

In [4]:
# reshape تصاویر از (32, 32, 3) به (3072,)
X_train = X_train.reshape((X_train.shape[0], -1))  # (50000, 3072)
X_test = X_test.reshape((X_test.shape[0], -1))     # (10000, 3072)

y_train, y_test = y_train.flatten(), y_test.flatten() # (50000,  -   10000,)

## **Model**

### **Activation Function : Sigmoid**

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Example:
print(sigmoid(10))
print(sigmoid(0))

0.9999546021312976
0.5
